# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import calendar

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.ensemble import EasyEnsembleClassifier

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
# Create the LabelEncoder instance
le = LabelEncoder()

# Fitting and encoding the columns with the LabelEncoder
def letransform (df, column):
    le.fit(df[column])
    df[column] = le.transform(df[column])
    return df[column]

# Label encoding columns
df["home_ownership"] = letransform(df, "home_ownership")
df["verification_status"] = letransform(df, "verification_status")
df["pymnt_plan"] = letransform(df, "pymnt_plan")
df["initial_list_status"] = letransform(df, "pymnt_plan")
df["application_type"] = letransform(df, "application_type")
df["hardship_flag"] = letransform(df, "hardship_flag")
df["debt_settlement_flag"] = letransform(df, "debt_settlement_flag")

# Splitting Issue_d column to Month and Year column 
df = pd.concat([df, df['issue_d'].str.partition('-', expand=True)[[0, 2]].rename({0: 'Month', 2: 'Year'}, axis=1)], axis=1)

# Months dictionary
name_to_num = {name: num for num, name in enumerate(calendar.month_abbr) if num}
name_to_num

#drop Nas
df.dropna()

# Encode month name
df["Month"] = df["Month"].apply(lambda x: name_to_num[x])

df.drop(columns="issue_d", inplace=True)

#Encoding Data using `get_dummies()`
df = pd.get_dummies(df, columns=["next_pymnt_d"])

#View the dataframe
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,Month,Year,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019
0,10500.0,0.1719,375.35,3,66000.0,1,low_risk,0,27.24,0.0,...,65687.0,38199.0,2000.0,61987.0,0,0,3,2019,0,1
1,25000.0,0.2000,929.09,1,105000.0,2,low_risk,0,20.23,0.0,...,271427.0,60641.0,41200.0,49197.0,0,0,3,2019,0,1
2,20000.0,0.2000,529.88,1,56000.0,2,low_risk,0,24.26,0.0,...,60644.0,45684.0,7500.0,43144.0,0,0,3,2019,0,1
3,10000.0,0.1640,353.55,3,92000.0,2,low_risk,0,31.44,0.0,...,99506.0,68784.0,19700.0,76506.0,0,0,3,2019,0,1
4,22000.0,0.1474,520.39,1,52000.0,0,low_risk,0,18.76,0.0,...,219750.0,25919.0,27600.0,20000.0,0,0,3,2019,0,1


## Split the Data into Training and Testing

In [6]:
df.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,Month,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,...,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.0,21.778153,0.217766,0.497697,...,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0,1.726172,0.383161,0.616839
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.0,20.199244,0.718367,0.758122,...,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0,0.743862,0.486161,0.486161
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0,1.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.0,13.890000,0.000000,0.000000,...,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0,1.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,0.0,19.760000,0.000000,0.000000,...,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0,2.000000,0.000000,1.000000
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,0.0,26.660000,0.000000,1.000000,...,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0,2.000000,1.000000,1.000000
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,0.0,999.000000,18.000000,5.000000,...,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0,3.000000,1.000000,1.000000


In [7]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]

In [8]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,Month,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,...,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.0,21.778153,0.217766,0.497697,...,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0,1.726172,0.383161,0.616839
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.0,20.199244,0.718367,0.758122,...,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0,0.743862,0.486161,0.486161
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0,1.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.0,13.890000,0.000000,0.000000,...,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0,1.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,0.0,19.760000,0.000000,0.000000,...,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0,2.000000,0.000000,1.000000
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,0.0,26.660000,0.000000,1.000000,...,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0,2.000000,1.000000,1.000000
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,0.0,999.000000,18.000000,5.000000,...,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0,3.000000,1.000000,1.000000


In [9]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [10]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X_train.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,Month,Year,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019
3903,1600.0,0.0881,50.74,2,35964.0,0,0,24.12,0.0,0.0,...,148200.0,14292.0,58300.0,15000.0,0,0,3,2019,0,1
28390,9000.0,0.0756,280.21,2,41000.0,0,0,16.89,1.0,0.0,...,68292.0,52587.0,24500.0,40492.0,0,0,2,2019,0,1
15470,10000.0,0.1033,214.10,1,112000.0,2,0,17.75,0.0,0.0,...,416023.0,78944.0,65400.0,80642.0,0,0,2,2019,0,1
4279,36000.0,0.1033,1167.21,3,120000.0,0,0,19.95,0.0,1.0,...,116200.0,73826.0,85200.0,30000.0,0,0,3,2019,0,1
57514,18000.0,0.0881,570.81,1,51000.0,2,0,19.11,1.0,0.0,...,221075.0,126195.0,36500.0,96715.0,0,0,1,2019,0,1


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [11]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [12]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [13]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [14]:
# Resample the training data with the BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [15]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7736940117070641

In [16]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_df

,Predicted 0,Predicted 1
Actual 0,65,36
Actual 1,1645,15459


In [17]:
# Print the imbalanced classification report
y_pred_rf = brf.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.64      0.90      0.07      0.76      0.57       101
   low_risk       1.00      0.90      0.64      0.95      0.76      0.60     17104

avg / total       0.99      0.90      0.65      0.94      0.76      0.60     17205



In [18]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
importances_sorted = sorted(zip(brf.feature_importances_, X.columns), reverse=True)
importances_sorted[:10]

[(0.06734465565213844, 'total_rec_prncp'),
 (0.06299024633299768, 'last_pymnt_amnt'),
 (0.06172965502018925, 'total_pymnt_inv'),
 (0.05862962492786315, 'total_pymnt'),
 (0.056087805054940536, 'total_rec_int'),
 (0.0319638217096265, 'int_rate'),
 (0.029531373890371186, 'Month'),
 (0.01848542379328983, 'mths_since_recent_inq'),
 (0.018448733700305874, 'dti'),
 (0.016274456927492836, 'out_prncp')]

### Easy Ensemble Classifier

In [19]:
# Train the Classifier
eec = EasyEnsembleClassifier(n_estimators=1000, random_state=1)
eec.fit(X_train, y_train)
X_test = scaler.transform(X_test)

In [20]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5

In [21]:
# Display the confusion matrix
cm_eec = confusion_matrix(y_test, y_pred)
cm_eec_df = pd.DataFrame(
    cm_eec, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_eec_df

,Predicted 0,Predicted 1
Actual 0,101,0
Actual 1,17104,0


In [ ]:
# Print the imbalanced classification report
y_pred_rf = eec.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred_rf))

# Final Questions

1. Which model had the best balanced accuracy score?

    Balanced Random Forest Classifier had the best balanced accuracy score of 77.3%

2. Which model had the best recall score?


3. Which model had the best geometric mean score?

    Balanced Random Forest Classifier had the best geo mean score of 76%

4. What are the top three features?
    
    1. total_rec_prncp
    2. last_pymnt_amnt
    3. total_pymnt_inv